In [1]:
import numpy as np
import pandas as pd
from collections import Counter
from datetime import datetime

In [2]:
class CandleStick:
    def __init__(self,date_time: str, open: float, high: float, low: float, close: float, volume: int=None):
        self.open = open
        self.high = high
        self.low = low
        self.close = close
        self.date_time = date_time
        self.volume = volume
        self._validate_candle()

    def __repr__(self):
        return f"Candle({self.date_time}, {self.open}, {self.high}, {self.low}, {self.close}, {self.volume})"

    def __str__(self):
        return f"DT: {self.date_time}, O: {self.open}, H: {self.high}, L: {self.low}, C: {self.close}, V: {self.volume}"

    @property
    def date_time(self) -> str:
        """
        forwards the date and time of the candlestick
        :return: str: date and time
        """
        return self._date_time

    @property
    def open(self) -> float:
        """
        forwards the open price of the candlestick
        :return: float: open price
        """
        return self._open

    @property
    def high(self) -> float:
        """
        forwards the high of the candlestick
        :return: float: high
        """
        return self._high

    @property
    def low(self) -> float:
        """
        forwards the low of the candlestick
        :return: float: low
        """
        return self._low

    @property
    def close(self) -> float:
        """
        forwards the close price of the candlestick
        :return: float: close price
        """
        return self._close

    @property
    def volume(self) -> int:
        """
        forwards the open Volume of the candlestick
        :return: int: Volume
        """
        return self._volume

    @date_time.setter
    def date_time(self, value: str) -> None:
        """
        method to validate the date_time
        :param value: str: date_time
        :return: None
        """
        if not isinstance(value, (str, datetime)):
            raise TypeError("date_time must be str or datetime not {}".format(type(value)))
        self._date_time = value
        return None

    @open.setter
    def open(self, value: float) -> None:
        """
        method to validate the open price
        :param value: float: open price
        :return: None
        """
        if not isinstance(value, (int, float)):
            raise TypeError("open must be int or float not {}".format(type(value)))
        if value < 0:
            raise ValueError("open must be positive")
        self._open = value
        return None

    @high.setter
    def high(self, value: float) -> None:
        """
        method to validate the high
        :param value: float: high
        :return: None
        """
        if not isinstance(value, (int, float)):
            raise TypeError("high must be int or float not {}".format(type(value)))
        if value < 0:
            raise ValueError("high must be positive")
        self._high = value
        return None

    @low.setter
    def low(self, value: float) -> None:
        """
        method to validate the low
        :param value: float: low
        :return: None
        """
        if not isinstance(value, (int, float)):
            raise TypeError("low must be int or float not {}".format(type(value)))
        if value < 0:
            raise ValueError("Low must be positive")
        self._low = value
        return None

    @close.setter
    def close(self, value: float) -> None:
        """
        method to validate the close price
        :param value: float: close price
        :return: None
        """
        if not isinstance(value, (int, float)):
            raise TypeError("close must be int or float not {}".format(type(value)))
        if value < 0:
            raise ValueError("close must be positive")
        self._close = value
        return None

    @volume.setter
    def volume(self, value: int) -> None:
        """
        method to validate the volume
        :param value: int: volume
        :return: None
        """
        if not isinstance(value, (int, float, type(None))):
            raise TypeError("volume must be int, float or NoneType not {}".format(type(value)))
        if value is not None:
            if value < 0:
                raise ValueError("volume must be positive")
        self._volume = value
        return None

    def _validate_candle(self) -> None:
        """
        method to validate the candlestick
        :return: None
        """
        if self.open > self.high:
            raise ValueError("open cannot be greater than high")
        if self.open < self.low:
            raise ValueError("open cannot be less than low")
        if self.close > self.high:
            raise ValueError("close cannot be greater than high")
        if self.close < self.low:
            raise ValueError("close cannot be less than low")

    def type(self) -> str:
        """
        method to determine the type of the candlestick
        :return: str: type of the candlestick
        """
        if self.open < self.close:
            return 'bullish'
        elif self.open > self.close:
            return 'bearish'
        else:
            return 'doji'

    def cs_size(self) -> float:
        """
        method to determine the size of the candlestick
        :return: float: size of the candlestick
        """
        return abs(self.high - self.low)

    def upper_shadow_size(self) -> float:
        """
        method to determine the size of the upper shadow
        :return: float: size of the upper shadow
        """
        return self.high - max(self.open, self.close)

    def lower_shadow_size(self) -> float:
        """
        method to determine the size of the lower shadow
        :return: float: size of the lower shadow
        """
        return min(self.open, self.close) - self.low

    def body_size(self) -> float:
        """
        method to determine the size of the body
        :return: float: size of the body
        """
        return abs(self.close - self.open)

    def is_bullish(self) -> bool:
        """
        method to determine if the candlestick is bullish
        :return: bool: True if bullish, False otherwise
        """
        return self.type() == 'bullish'

    def is_bearish(self) -> bool:
        """
        method to determine if the candlestick is bearish
        :return: bool: True if bearish, False otherwise
        """
        return self.type() == 'bearish'

    def is_doji(self) -> bool:
        """
        method to determine if the candlestick is a doji
        :return: bool: True if doji, False otherwise
        """
        return self.type() == 'doji'

    def cs_body_ratio(self) -> float:
        """
        method to determine the ratio of the body to the candlestick
        :return: float: ratio of the body to the candlestick, range [0, 1]
        """
        return self.body_size() / self.cs_size() if self.cs_size() > 0 else 0

    def body_upper_shadow_ratio(self) -> float:
        """
        method to determine the ratio of the upper shadow to the body
        can be between 0 and infinity if the body is 0 (doji)
        shows how big the upper shadow is compared to the body size
        :return: float: ratio of the upper shadow to the body, range [0, ∞]
        """
        return self.upper_shadow_size() / self.body_size() if self.body_size() > 0 else np.inf

    def body_lower_shadow_ratio(self) -> float:
        """
        method to determine the ratio of the lower shadow to the body
        can be between 0 and infinity if the body is 0 (doji)
        shows how big the lower shadow is compared to the body size
        :return: float: ratio of the lower shadow to the body, range [0, ∞)
        """
        return self.lower_shadow_size() / self.body_size() if self.body_size() > 0 else np.inf

    def body_position(self) -> float:
        """
        method to determine the position of the body
        can be between -1 and 1
        -1 = body totally at the bottom of cs, 0 = middle, 1 = body totally at the top of cs
        :return: float: position of the body, range [-1, 1]
        """
        upper_shadow = self.high - max(self.open, self.close)
        lower_shadow = min(self.open, self.close) - self.low
        shadows = upper_shadow + lower_shadow
        return  (2 * lower_shadow) / shadows -1 if shadows != 0 else 0 # -1 = lower shadow, 0 = middle, 1 = upper shadow

In [3]:
class CandleStickFrame:
    def __init__(self,date_time:list, open: list, high: list, low: list, close: list, volume: list = None):
        date_time, open, high, low, close, volume = self._validate_input(date_time, open, high, low, close, volume)
        self.candle_sticks = [CandleStick(dt, o, h, l, c, v) for dt, o, h, l, c, v in zip(date_time, open, high, low, close, volume)]
        self.df = pd.DataFrame({"open": open, "high": high, "low": low, "close": close})
        self._bullish_count, self._bearish_count, self._doji_count = self._type_count()

    def __repr__(self):
        return f"CandleFrame({self.df})"

    def __str__(self):
        return f"{self.df}"

    def __len__(self):
        return len(self.candle_sticks)

    def __getitem__(self, index):
        return self.candle_sticks[index]

    def __iter__(self):
        return iter(self.candle_sticks)

    def __reversed__(self):
        return reversed(self.candle_sticks)

    @staticmethod
    def _validate_input(date_time: str, open: float, high: float, low: float, close: float, volume: int) -> tuple:
        """
        method to validate the input
        :param date_time: list, np.ndarray, or pd.core.series.Series of date_time of the candlesticks
        :param open: list, np.ndarray, or pd.core.series.Series of open of the candlesticks
        :param high: list, np.ndarray, or pd.core.series.Series of high of the candlesticks
        :param low: list, np.ndarray, or pd.core.series.Series of low of the candlesticks
        :param close: list, np.ndarray, or pd.core.series.Series of close of the candlesticks
        :param volume: list, np.ndarray, or pd.core.series.Series of volume of the candlesticks or None
        :return: touple: date_time, open, high, low, close, volume
        """
        if not isinstance(date_time, (list, np.ndarray, pd.core.series.Series)):
            raise TypeError("date_time must be list, np.ndarry, or pd.core.series.Series not {}".format(type(date_time)))
        if not isinstance(open, (list, np.ndarray, pd.core.series.Series)):
            raise TypeError("open must be list, np.ndarry, or pd.core.series.Series not {}".format(type(open)))
        if not isinstance(high, (list, np.ndarray, pd.core.series.Series)):
            raise TypeError("high must be list, np.ndarry, or pd.core.series.Series not {}".format(type(high)))
        if not isinstance(low, (list, np.ndarray, pd.core.series.Series)):
            raise TypeError("low must be list, np.ndarry, or pd.core.series.Series not {}".format(type(low)))
        if not isinstance(close, (list, np.ndarray, pd.core.series.Series)):
            raise TypeError("close must be list, np.ndarry, or pd.core.series.Series not {}".format(type(close)))
        if not isinstance(volume, (list, np.ndarray, pd.core.series.Series, type(None))):
            raise TypeError("volume must be list, np.ndarry, pd.core.series.Series or NoneType not {}".format(type(volume)))
        date_time = list(date_time)
        open = list(open)
        high = list(high)
        low = list(low)
        close = list(close)
        volume = list(volume) if volume is not None else [None] * len(date_time)
        if len(date_time) != len(open) != len(high) or len(open) != len(low) or len(open) != len(close):
            raise ValueError("date_time, open, high, low, and close must be the same length")
        if not all(isinstance(x, (str, datetime)) for x in date_time):
            raise TypeError("date_time must be list of str or time.datetime not {}".format(type(date_time)))
        if not all(isinstance(x, (int, float)) for x in open):
            raise TypeError("open must be list of int or float not {}".format(type(open)))
        if not all(isinstance(x, (int, float)) for x in high):
            raise TypeError("high must be list of int or float not {}".format(type(high)))
        if not all(isinstance(x, (int, float)) for x in low):
            raise TypeError("low must be list of int or float not {}".format(type(low)))
        if not all(isinstance(x, (int, float)) for x in close):
            raise TypeError("close must be list of int or float not {}".format(type(close)))
        if not all(isinstance(x, (int, float, type(None))) for x in volume):
            raise TypeError("volume must be list of int, float or None not {}".format(type(volume)))
        return date_time, open, high, low, close, volume

    def _type_count(self) -> tuple:
        """
        method to count the number of bullish, bearish, and doji candlesticks
        :return: touple: bullish, bearish, doji count
        """
        bullish = 0
        bearish = 0
        doji = 0
        for candle in self.candle_sticks:
            if candle.type() == 'bullish':
                bullish += 1
            elif candle.type() == 'bearish':
                bearish += 1
            elif candle.type() == 'doji':
                doji += 1
        return bullish, bearish, doji

    def _bullish_ratio(self) -> float:
        """
        method to calculate the ratio of bullish candlesticks
        :return: float: bullish ratio range [0, 1]
        """
        return self._bullish_count / len(self.candle_sticks)

    def _bearish_ratio(self) -> float:
        """
        method to calculate the ratio of bearish candlesticks
        :return: float: bearish ratio range [0, 1]
        """
        return self._bearish_count / len(self.candle_sticks)

    def _doji_ratio(self) -> float:
        """
        method to calculate the ratio of doji candlesticks
        :return: float: doji ratio range [0, 1]
        """
        return self._doji_count / len(self.candle_sticks)

    def type_ratio(self) -> str:
        """
        method to return the ratio of bullish, bearish, and doji candlesticks
        :return: str: bullish, bearish, and doji ratio
        """
        return 'bullish: {:.2f}, bearish: {:.2f}, doji: {:.2f}'.format(self._bullish_ratio(), self._bearish_ratio(), self._doji_ratio())

In [4]:
class Parameter:
    candle_stick_pattern = dict(
        bullish=dict(
            hammer = dict(
                trend_window = 10, # len of trend window [1, inf]
                trend_strength = -0.00, # to be valid trend must be less than this value [-1, 0]
                cs_body_position = 0.25, # to be valid body must have a position greater than this value [0, 1]
                body_ls_ratio = 1.75, # to be valid lower shadow must be greater than this value compared to the body [0, inf]
                body_us_ratio = 0.50, # to be valid upper shadow must be less than this value compared to the body [0, inf]
            ),
            piercing = dict(
                trend_window = 10, # len of trend window [1, inf]
                trend_strength = -0.00, # to be valid trend must be less than this value [-1, 0]
            ),
            bullish_engulfing = dict(
                trend_window = 10, # len of trend window [1, inf]
                trend_strength = -0.00, # to be valid trend must be less than this value [-1, 0]
            ),
            morning_star = dict(
                trend_window = 10, # len of trend window [1, inf]
                relative_size_window = 11, # len of relative size window [1, inf]
                trend_strength = -0.00, # to be valid trend must be less than this value [-1, 0]
                cs_body_ratio = 0.50, # to be valid body must be greater than this value compared to the cs [0, 1]
                cs_m1_body_ratio = 0.50, # to be valid body must be less than this value compared to the cs [0, 1]
                cs_m2_body_ratio = 0.50, # to be valid body must be greater than this value compared to the cs [0, 1]
                cs_relative_size = 0.30, # to be valid relative size must be greater than this value [0, 1]
                cs_m2_relative_size = 0.30, # to be valid relative size must be greater than this value [0, 1]
            ),
            three_white_soldiers = dict(
                trend_window = 10, # len of trend window [1, inf]
                relative_size_window = 11, # len of relative size window [1, inf]
                trend_strength = -0.00, # to be valid trend must be less than this value [-1, 0]
                cs_relative_size = 0.25, # to be valid relative size must be greater than this value [0, 1]
                cs_m1_relative_size = 0.25, # to be valid relative size must be greater than this value [0, 1]
                cs_m2_relative_size = 0.25, # to be valid relative size must be greater than this value [0, 1]
                cs_body_ratio = 0.30, # to be valid body must be greater than this value compared to the cs [0, 1]
                cs_m1_body_ratio = 0.30, # to be valid body must be greater than this value compared to the cs [0, 1]
                cs_m2_body_ratio = 0.30, # to be valid body must be greater than this value compared to the cs [0, 1]
            ),
            bullish_marubozu = dict(
                trend_window = 10, # len of trend window [1, inf]
                relative_size_window = 11, # len of relative size window [1, inf]
                trend_strength = -0.00, # to be valid trend must be less than this value [-1, 0]
                cs_relative_size = 0.80, # to be valid relative size must be greater than this value [0, 1]
                cs_body_ratio = 0.80, # to be valid body must be greater than this value compared to the cs [0, 1]
            ),
            three_inside_up = dict(
                trend_window = 10,
                relative_size_window = 11,
                trend_strength = 0.00,
                cs_m2_body_ratio = 0.50,
                cs_body_ratio = 0.50,
                cs_m2_relative_size = 0.30,
                cs_relative_size = 0.30,
            ),
            bullish_harami = dict(
                trend_window = 10,
                relative_size_window = 11,
                trend_strength = 0.00,
                cs_m1_body_ratio = 0.50,
                cs_body_ratio = 0.50,
                cs_m1_relative_size = 0.30,
                cs_relative_size = 0.70,
            ),
            tweezer_bottom = dict(
                trend_window = 10,
                relative_size_window = 11,
                trend_strength = 0.00,
                cs_m1_body_ratio = 0.50,
                cs_body_ratio = 0.50,
                cs_m1_relative_size = 0.30,
                cs_body_position = -0.25,
            )
        ),
        bearish=dict(
            hanging_man = dict(
                trend_window = 10, # len of trend window [1, inf]
                trend_strength = 0.00, # to be valid trend must be greater than this value [0, 1]
                cs_body_position = -0.25, # to be valid body must have a position less than this value [-1, 0]
                body_ls_ratio = 0.50, # to be valid lower shadow must be less than this value compared to the body [0, inf]
                body_us_ratio = 1.50, # to be valid upper shadow must be greater than this value compared to the body [0, inf]
            ),
            dark_cloud = dict(
                trend_window = 10, # len of trend window [1, inf]
                trend_strength = 0.00, # to be valid trend must be greater than this value [0, 1]
            ),
            bearish_engulfing = dict(
                trend_window = 10, # len of trend window [1, inf]
                trend_strength = 0.00, # to be valid trend must be greater than this value [0, 1]
            ),
            evening_star = dict(
                trend_window = 10, # len of trend window [1, inf]
                relative_size_window = 11, # len of relative size window [1, inf]
                trend_strength = 0.00, # to be valid trend must be greater than this value [0, 1]
                cs_body_ratio = 0.50, # to be valid body must be greater than this value compared to the cs [0, 1]
                cs_m1_body_ratio = 0.50, # to be valid body must be less than this value compared to the cs [0, 1]
                cs_m2_body_ratio = 0.50, # to be valid body must be greater than this value compared to the cs [0, 1]
                cs_relative_size = 0.30, # to be valid relative size must be greater than this value [0, 1]
                cs_m2_relative_size = 0.30, # to be valid relative size must be greater than this value [0, 1]
            ),
            three_black_crows = dict(
                trend_window = 10, # len of trend window [1, inf]
                relative_size_window = 11, # len of relative size window [1, inf]
                trend_strength = -0.00, # to be valid trend must be less than this value [-1, 0]
                cs_relative_size = 0.25, # to be valid relative size must be greater than this value [0, 1]
                cs_m1_relative_size = 0.25, # to be valid relative size must be greater than this value [0, 1]
                cs_m2_relative_size = 0.25, # to be valid relative size must be greater than this value [0, 1]
                cs_body_ratio = 0.30, # to be valid body must be greater than this value compared to the cs [0, 1]
                cs_m1_body_ratio = 0.30, # to be valid body must be greater than this value compared to the cs [0, 1]
                cs_m2_body_ratio = 0.30, # to be valid body must be greater than this value compared to the cs [0, 1]
            ),
            bearish_marubozu = dict(
                trend_window = 10, # len of trend window [1, inf]
                relative_size_window = 11, # len of relative size window [1, inf]
                trend_strength = 0.00, # to be valid trend must be greater than this value [0, 1]
                cs_relative_size = 0.80, # to be valid relative size must be greater than this value [0, 1]
                cs_body_ratio = 0.80, # to be valid body must be greater than this value compared to the cs [0, 1]
            ),
            three_inside_down = dict(
                trend_window = 10,
                relative_size_window = 11,
                trend_strength = 0.00,
                cs_m2_body_ratio = 0.50,
                cs_body_ratio = 0.50,
                cs_m2_relative_size = 0.30,
                cs_relative_size = 0.30,
            ),
            bearish_harami = dict(
                trend_window = 10,
                relative_size_window = 11,
                trend_strength = 0.00,
                cs_m1_body_ratio = 0.50,
                cs_body_ratio = 0.50,
                cs_m1_relative_size = 0.30,
                cs_relative_size = 0.70,
            ),
            tweezer_top = dict(
                trend_window = 10,
                relative_size_window = 11,
                trend_strength = 0.00,
                cs_m1_body_ratio = 0.50,
                cs_body_ratio = 0.50,
                cs_m1_relative_size = 0.30,
                cs_body_position = 0.25,
            )
        ),
    )

In [5]:
class CandleStickPattern:
    def __init__(self, candle_stick_frame: CandleStickFrame):
        self.candle_stick_frame = self._validate_csf(candle_stick_frame)

    def _validate_csf(self, candle_stick_frame: CandleStickFrame):
        """
        method to validate the candle stick frame
        :param candle_stick_frame: CandleStickFrame: candle stick frame to validate
        :return: CandleStickFrame: validated candle stick frame
        """
        if not isinstance(candle_stick_frame, CandleStickFrame):
            raise TypeError("candle_stick_frame must be CandleStickFrame not {}".format(type(candle_stick_frame)))
        if len(candle_stick_frame) < 1:
            raise ValueError("candle_stick_frame must have at least 1 candle stick")
        return candle_stick_frame

    def _prepare_window(self, index:int, window: int) -> CandleStickFrame:
        """
        method to prepare the window for the candle stick frame
        :param index: int: index of the candle stick
        :param window: int: window to look back
        :return: CandleStickFrame: window of candle sticks
        """
        if window < 1:
            raise ValueError("window must be greater than 0")
        if window > len(self.candle_stick_frame):
            raise ValueError("window must be less than or equal to the length of the candle stick frame")
        return self.candle_stick_frame[index-window:index]

    def z_score_cs(self, index: int, window: int=10) -> float:
        """
        method to calculate the z score of the candle sticks in the window
        cs size normal scaled in respect to the cs of the candle sticks in the window
        can be in range [-inf, inf] with a mean of 0 and standard deviation of 1
        :param index: int: index of the candle stick to calculate the z score of
        :param window: int: window to calculate the z score over
        :return: list or None: z score of the candle sticks of the window [-inf, inf]
        """
        cs_window = self._prepare_window(index, window)
        if len(cs_window) < window or index < 0:
            return None
        else:
            mean = np.mean([cs.cs_size() for cs in cs_window])
            std = np.std([cs.cs_size() for cs in cs_window])
            # cs size normal scaled in respect to the cs of the candle sticks in the window
            scaled_cs = list(map(lambda cs: (cs.cs_size() - mean) / std, cs_window))
            return scaled_cs

    def z_score_body(self, index: int, window: int=11) -> float:
        """
        method to calculate the z score of the bodies in the window
        body size normal scaled in respect to the bodies of the candle sticks in the window
        can be in range [-inf, inf] with a mean of 0 and standard deviation of 1
        :param index: int: index of the candle stick to calculate the z score of
        :param window: int: window to calculate the z score over
        :return: list or None: z score of the bodies of the window [-inf, inf]
        """
        # the current cs should be included in the window -> += 1
        index += 1
        cs_window = self._prepare_window(index, window)
        if len(cs_window) < window or index < 0:
            return None
        else:
            mean = np.mean([cs.body_size() for cs in cs_window])
            std = np.std([cs.body_size() for cs in cs_window])
            # body size normal scaled in respect to the bodies of the candle sticks in the window
            scaled_bodies = list(map(lambda cs: (cs.body_size() - mean) / std, cs_window))
            return scaled_bodies

    def min_max_cs(self, index: int, window: int=11) -> float:
        """
        method to calculate the min max of the candle sticks in the window
        cs size min max scaled in respect to the cs of the candle sticks in the window
        can be in range [0, 1]
        :param index: int: index of the candle stick to calculate the min max of
        :param window: int: window to calculate the min max over
        :return: list or None: min max of the candle sticks in the window [0, 1]
        """
        # the current cs should be included in the window -> += 1
        index += 1
        cs_window = self._prepare_window(index, window)
        if len(cs_window) < window or index < 0:
            return None
        else:
            min = np.min([cs.cs_size() for cs in cs_window])
            max = np.max([cs.cs_size() for cs in cs_window])
            # cs size min max scaled in respect to the cs of the candle sticks in the window
            scaled_cs = list(map(lambda cs: (cs.cs_size() - min) / (max - min), cs_window))
            return scaled_cs

    def min_max_body(self, index: int, window: int=11) -> float:
        """
        method to calculate the min max of the bodies in the window
        body size min max scaled in respect to the bodies of the candle sticks in the window
        can be in range [0, 1]
        :param index: int: index of the candle stick to calculate the min max of
        :param window: int: window to calculate the min max over
        :return: list or None: min max of the bodies in the Window [0, 1]
        """
        # the current cs should be included in the window -> += 1
        index += 1
        cs_window = self._prepare_window(index, window)
        if len(cs_window) < window or index < 0:
            return None
        else:
            min = np.min([cs.body_size() for cs in cs_window])
            max = np.max([cs.body_size() for cs in cs_window])
            # body size min max scaled in respect to the bodies of the candle sticks in the window
            scaled_bodies = list(map(lambda cs: (cs.body_size() - min) / (max - min), cs_window))
            return scaled_bodies

    def trend(self, index: int, window:int=10) -> float:
        """
        method to calculate the trend of the candle stick at index
        can be in range [-1, 1] with -1 being down and 1 being up
        :param index: int: index of the candle stick to calculate the trend of
        :param window: int: window to calculate the trend over
        :return: float or None: trend of the candle stick at index [-1, 1]
        """
        cs_window = self._prepare_window(index, window)
        if len(cs_window) < window or index < 0:
            return None
        trend, sum = 0, 0
        for cs in cs_window:
            sum += cs.cs_size()
            if cs.type() == 'bullish':
                trend += cs.cs_size()
            elif cs.type() == 'bearish':
                trend -= cs.cs_size()
        # weighted average of the trend
        return trend / sum

    class PatternTemplate:
        def __init__(self, param: dict, pattern_name: str, pattern_type: str, trend_strength: float, pattern: list):
            self.param = param
            self.pattern_name = pattern_name
            self.pattern_type = pattern_type
            self.trend_strength = trend_strength
            self.pattern = pattern
            self.is_pattern = False

        def __str__(self):
            return f'{self.pattern_name} -> ({self.is_pattern})'

        def __repr__(self):
            return f'{self.pattern_name} -> ({self.is_pattern})'


    # Bullish Reversal Candlestick Patterns classes:
    # Hammer (1)
    class Hammer(PatternTemplate):
        def __init__(self, trend: float, cs: CandleStick, param:dict):
            super().__init__(param, 'Hammer', 'bullish', trend, [cs])
            self._cs = cs
            self.is_pattern = self._is_hammer()

        def _is_hammer(self) -> bool:
            """
            method to check if the candle stick is a hammer
            :return: bool: True if the candle stick is a hammer, False otherwise
            """
            if self.trend_strength is None:
                return None
            if self.trend_strength <= self.param['trend_strength']:
                if self._cs.type() == 'bullish' and self._cs.body_position() >= self.param['cs_body_position'] and self._cs.body_lower_shadow_ratio() >= self.param['body_ls_ratio'] and self._cs.body_upper_shadow_ratio() <= self.param['body_us_ratio']:
                    return True
            return False

    # Bullish Piercing (2)
    class Piercing(PatternTemplate):
        def __init__(self, trend: float, cs: CandleStick, cs_m1: CandleStick, param:dict):
            super().__init__(param, 'Piercing', 'bullish', trend, [cs_m1, cs])
            self._cs = cs
            self._cs_m1 = cs_m1
            self.is_pattern = self._is_piercing()

        def _is_piercing(self) -> bool:
            """
            method to check if the candle stick is a piercing
            :return: bool: True if the candle stick is a piercing, False otherwise
            """
            if self.trend_strength is None:
                return None
            if self.trend_strength <= self.param['trend_strength']:
                if self._cs.type() == 'bullish' and self._cs_m1.type() == 'bearish':
                    if self._cs.open < self._cs_m1.close and self._cs_m1.open - self._cs_m1.body_size() / 2 < self._cs.close < self._cs_m1.open:
                        return True
            return False

    # Bullish Engulfing (3)
    class BullishEngulfing(PatternTemplate):
        def __init__(self, trend: float, cs: CandleStick, cs_m1: CandleStick, param:dict):
            super().__init__(param, 'BullishEngulfing', 'bullish', trend, [cs_m1, cs])
            self._cs = cs
            self._cs_m1 = cs_m1
            self.is_pattern = self._is_bullish_engulfing()

        def _is_bullish_engulfing(self) -> bool:
            """
            method to check if the candle stick is a bullish engulfing
            :return: bool: True if the candle stick is a bullish engulfing, False otherwise
            """
            if self.trend_strength is None:
                return None
            if self.trend_strength <= self.param['trend_strength']:
                if self._cs.type() == 'bullish' and self._cs_m1.type() == 'bearish':
                    if self._cs.open < self._cs_m1.close and self._cs.close > self._cs_m1.open:
                        return True
            return False

    # Morning Star (4)
    class MorningStar(PatternTemplate):
        def __init__(self, trend: float, relative_size: list, cs: CandleStick, cs_m1: CandleStick, cs_m2: CandleStick, param:dict):
            super().__init__(param, 'MorningStar', 'bullish', trend, [cs_m2, cs_m1, cs])
            self._relative_size = relative_size
            self._cs = cs
            self._cs_m1 = cs_m1
            self._cs_m2 = cs_m2
            self.is_pattern = self._is_morning_star()

        def _is_morning_star(self) -> bool:
            """
            method to check if the candle stick is a morning star
            :return: bool: True if the candle stick is a morning star, False otherwise
            """
            if self.trend_strength is None:
                return None
            if self.trend_strength <= self.param['trend_strength']:
                if self._cs_m2.type() == 'bearish' and self._cs_m2.cs_body_ratio() >= self.param['cs_m2_body_ratio'] and self._relative_size[-3] >= self.param['cs_m2_relative_size']:
                    if self._cs_m1.cs_body_ratio() <= self.param['cs_m1_body_ratio']:
                        if self._cs.type() == 'bullish' and self._cs.cs_body_ratio() >= self.param['cs_body_ratio'] and self._relative_size[-1] >= self.param['cs_relative_size']:
                            return True
            return False

    # Three White Soldiers (5)
    class ThreeWhiteSoldiers(PatternTemplate):
        def __init__(self, trend: float, relative_size: list, cs: CandleStick, cs_m1: CandleStick, cs_m2: CandleStick, param:dict):
            super().__init__(param, 'ThreeWhiteSoldiers', 'bullish', trend, [cs_m2, cs_m1, cs])
            self._relative_size = relative_size
            self._cs = cs
            self._cs_m1 = cs_m1
            self._cs_m2 = cs_m2
            self.is_pattern = self._is_three_white_soldiers()

        def _is_three_white_soldiers(self) -> bool:
            """
            method to check if the candle stick is a three white soldiers
            :return: bool: True if the candle stick is a three white soldiers, False otherwise
            """
            if self.trend_strength is None:
                return None
            if self.trend_strength <= self.param['trend_strength']:
                if self._cs_m2.type() == 'bullish' and self._cs_m2.cs_body_ratio() >= self.param['cs_m2_body_ratio'] and self._relative_size[-3] >= self.param['cs_m2_relative_size']:
                    if self._cs_m1.type() == 'bullish' and self._cs_m1.cs_body_ratio() >= self.param['cs_m1_body_ratio'] and self._relative_size[-2] >= self.param['cs_m1_relative_size']:
                        if self._cs.type() == 'bullish' and self._cs.cs_body_ratio() >= self.param['cs_body_ratio'] and self._relative_size[-1] >= self.param['cs_relative_size']:
                            return True
            return False

    # Bullish Marubozu (6)
    class BullishMarubozu(PatternTemplate):
        def __init__(self, trend: float, relative_size: list, cs: CandleStick, param:dict):
            super().__init__(param, 'BullishMarubozu', 'bullish', trend, [cs])
            self._relative_size = relative_size
            self._cs = cs
            self.is_pattern = self._is_bullish_marubozu()

        def _is_bullish_marubozu(self) -> bool:
            """
            method to check if the candle stick is a bullish marubozu
            :return: bool: True if the candle stick is a bullish marubozu, False otherwise
            """
            if self.trend_strength is None:
                return None
            if self.trend_strength <= self.param['trend_strength']:
                if self._cs.type() == 'bullish' and self._cs.cs_body_ratio() >= self.param['cs_body_ratio'] and self._relative_size[-1] >= self.param['cs_relative_size']:
                    return True
            return False

    # Three Inside Up (7)
    class ThreeInsideUp(PatternTemplate):
        def __init__(self, trend: float, relative_size: list, cs: CandleStick, cs_m1: CandleStick, cs_m2: CandleStick, param:dict):
            super().__init__(param, 'ThreeInsideUp', 'bullish', trend, [cs_m2, cs_m1, cs])
            self._relative_size = relative_size
            self._cs = cs
            self._cs_m1 = cs_m1
            self._cs_m2 = cs_m2
            self.is_pattern = self._is_three_inside_up()

        def _is_three_inside_up(self) -> bool:
            """
            method to check if the candle stick is a three inside up
            :return: bool: True if the candle stick is a three inside up, False otherwise
            """
            if self.trend_strength is None:
                return None
            if self.trend_strength <= self.param['trend_strength']:
                if self._cs_m2.type() == 'bearish' and self._cs_m2.cs_body_ratio() >= self.param['cs_m2_body_ratio'] and self._relative_size[-3] >= self.param['cs_m2_relative_size']:
                    if self._cs_m1.type() == 'bullish' and self._cs_m1.open <= self._cs_m2.close:
                        if self._cs.type() == 'bullish' and self._cs.cs_body_ratio() >= self.param['cs_body_ratio'] and self._relative_size[-1] >= self.param['cs_relative_size'] and self._cs.open >= self._cs_m1.close:
                            return True
            return False

    # Bullish Harami (8)
    class BullishHarami(PatternTemplate):
        def __init__(self, trend: float, relative_size: list, cs: CandleStick, cs_m1: CandleStick, param:dict):
            super().__init__(param, 'BullishHarami', 'bullish', trend, [cs_m1, cs])
            self._relative_size = relative_size
            self._cs = cs
            self._cs_m1 = cs_m1
            self.is_pattern = self._is_bullish_harami()

        def _is_bullish_harami(self) -> bool:
            """
            method to check if the candle stick is a bullish harami
            :return: bool: True if the candle stick is a bullish harami, False otherwise
            """
            if self.trend_strength is None:
                return None
            if self.trend_strength <= self.param['trend_strength']:
                if self._cs_m1.type() == 'bearish' and self._cs_m1.cs_body_ratio() >= self.param['cs_m1_body_ratio'] and self._relative_size[-2] >= self.param['cs_m1_relative_size']:
                    if self._cs.type() == 'bullish' and self._cs.cs_body_ratio() >= self.param['cs_body_ratio'] and self._relative_size[-1] <= self.param['cs_relative_size'] and self._cs.open >= self._cs_m1.close and self._cs.close <= self._cs_m1.open:
                        return True
            return False

    # Tweezer Bottom (9)
    class TweezerBottom(PatternTemplate):
        def __init__(self, trend: float, relative_size: list, cs: CandleStick, cs_m1: CandleStick, param:dict):
            super().__init__(param, 'TweezerBottom', 'bullish', trend, [cs, cs_m1])
            self._relative_size = relative_size
            self._cs = cs
            self._cs_m1 = cs_m1
            self.is_pattern = self._is_tweezer_bottom()

        def _is_tweezer_bottom(self) -> bool:
            """
            method to check if the candle stick is a tweezer bottom
            :return: bool: True if the candle stick is a tweezer bottom, False otherwise
            """
            if self.trend_strength is None:
                return None
            if self.trend_strength <= self.param['trend_strength']:
                if self._cs_m1.type() == 'bearish' and self._cs_m1.cs_body_ratio() >= self.param['cs_m1_body_ratio'] and self._relative_size[-2] >= self.param['cs_m1_relative_size']:
                    if self._cs.type() == 'bullish' and self._cs.body_position() <= self.param['cs_body_position'] and self._cs.cs_body_ratio() <= self.param['cs_body_ratio']:
                        return True
            return False


    # Bearish Reversal Candlestick Patterns Classes:
    # Hanging Man (14)
    class HangingMan(PatternTemplate):
        def __init__(self, trend: float, cs: CandleStick, param:dict):
            super().__init__(param, 'HangingMan', 'bearish', trend, [cs])
            self._cs = cs
            self.is_pattern = self._is_hanging_man()

        def _is_hanging_man(self) -> bool:
            """
            method to check if the candle stick is a hanging man
            :return: bool: True if the candle stick is a hanging man, False otherwise
            """
            if self.trend_strength is None:
                return None
            if self.trend_strength >= self.param['trend_strength']:
                if self._cs.type() == 'bearish' and self._cs.body_position() <= self.param['cs_body_position'] and self._cs.body_lower_shadow_ratio() <= self.param['body_ls_ratio'] and self._cs.body_upper_shadow_ratio() >= self.param['body_us_ratio']:
                    return True
            return False

    # Dark Cloud (15)
    class DarkCloud(PatternTemplate):
        def __init__(self, trend: float, cs: CandleStick, cs_m1: CandleStick, param:dict):
            super().__init__(param, 'DarkCloud', 'bearish', trend, [cs_m1, cs])
            self._cs = cs
            self._cs_m1 = cs_m1
            self.is_pattern = self._is_dark_cloud()

        def _is_dark_cloud(self) -> bool:
            """
            method to check if the candle stick is a dark cloud
            :return: bool: True if the candle stick is a dark cloud, False otherwise
            """
            if self.trend_strength is None:
                return None
            if self.trend_strength >= self.param['trend_strength']:
                if self._cs.type() == 'bearish' and self._cs_m1.type() == 'bullish':
                    if self._cs.open > self._cs_m1.close and self._cs_m1.open + self._cs_m1.body_size() / 2 > self._cs.close > self._cs_m1.open:
                        return True
            return False

    # Bearish Engulfing (16)
    class BearishEngulfing(PatternTemplate):
        def __init__(self, trend: float, cs: CandleStick, cs_m1: CandleStick, param:dict):
            super().__init__(param, 'BearishEngulfing', 'bearish', trend, [cs_m1, cs])
            self._cs = cs
            self._cs_m1 = cs_m1
            self.is_pattern = self._is_bearish_engulfing()

        def _is_bearish_engulfing(self) -> bool:
            """
            method to check if the candle stick is a bearish engulfing
            :return: bool: True if the candle stick is a bearish engulfing, False otherwise
            """
            if self.trend_strength is None:
                return None
            if self.trend_strength >= self.param['trend_strength']:
                if self._cs.type() == 'bearish' and self._cs_m1.type() == 'bullish':
                    if self._cs.open > self._cs_m1.close and self._cs.close < self._cs_m1.open:
                        return True
            return False

    # Evening Star (17)
    class EveningStar(PatternTemplate):
        def __init__(self, trend: float, relative_size: list, cs: CandleStick, cs_m1: CandleStick, cs_m2: CandleStick, param:dict):
            super().__init__(param, 'EveningStar', 'bearish', trend, [cs_m2, cs_m1, cs])
            self._relative_size = relative_size
            self._cs = cs
            self._cs_m1 = cs_m1
            self._cs_m2 = cs_m2
            self.is_pattern = self._is_evening_star()

        def _is_evening_star(self) -> bool:
            """
            method to check if the candle stick is an evening star
            :return: bool: True if the candle stick is an evening star, False otherwise
            """
            if self.trend_strength is None:
                return None
            if self.trend_strength >= self.param['trend_strength']:
                if self._cs_m2.type() == 'bullish' and self._cs_m2.cs_body_ratio() >= self.param['cs_m2_body_ratio'] and self._relative_size[-3] >= self.param['cs_m2_relative_size']:
                    if self._cs_m1.cs_body_ratio() <= self.param['cs_m1_body_ratio']:
                        if self._cs.type() == 'bearish' and self._cs.cs_body_ratio() >= self.param['cs_body_ratio'] and self._relative_size[-1] >= self.param['cs_relative_size']:
                            return True
            return False

    # Three Black Crows (18)
    class ThreeBlackCrows(PatternTemplate):
        def __init__(self, trend: float, relative_size: list, cs: CandleStick, cs_m1: CandleStick, cs_m2: CandleStick, param:dict):
            super().__init__(param, 'ThreeBlackCrows', 'bearish', trend, [cs_m2, cs_m1, cs])
            self._relative_size = relative_size
            self._cs = cs
            self._cs_m1 = cs_m1
            self._cs_m2 = cs_m2
            self.is_pattern = self._is_three_black_crows()

        def _is_three_black_crows(self) -> bool:
            """
            method to check if the candle stick is a three black crows
            :return: bool: True if the candle stick is a three black crows, False otherwise
            """
            if self.trend_strength is None:
                return None
            if self.trend_strength >= self.param['trend_strength']:
                if self._cs_m2.type() == 'bearish' and self._cs_m2.cs_body_ratio() >= self.param['cs_m2_body_ratio'] and self._relative_size[-3] >= self.param['cs_m2_relative_size']:
                    if self._cs_m1.type() == 'bearish' and self._cs_m1.cs_body_ratio() >= self.param['cs_m1_body_ratio'] and self._relative_size[-2] >= self.param['cs_m1_relative_size']:
                        if self._cs.type() == 'bearish' and self._cs.cs_body_ratio() >= self.param['cs_body_ratio'] and self._relative_size[-1] >= self.param['cs_relative_size']:
                            return True
            return False

    # Bearish Marubozu (19)
    class BearishMarubozu(PatternTemplate):
        def __init__(self, trend: float, relative_size: list, cs: CandleStick, param:dict):
            super().__init__(param, 'BearishMarubozu', 'bearish', trend, [cs])
            self._relative_size = relative_size
            self._cs = cs
            self.is_pattern = self._is_bearish_marubozu()

        def _is_bearish_marubozu(self) -> bool:
            """
            method to check if the candle stick is a bearish marubozu
            :return: bool: True if the candle stick is a bearish marubozu, False otherwise
            """
            if self.trend_strength is None:
                return None
            if self.trend_strength >= self.param['trend_strength']:
                if self._cs.is_bearish() and self._cs.cs_body_ratio() >= self.param['cs_body_ratio'] and self._relative_size[-1] >= self.param['cs_relative_size']:
                    return True
            return False

    # Three Inside Down (20)
    class ThreeInsideDown(PatternTemplate):
        def __init__(self, trend: float, relative_size: list, cs: CandleStick, cs_m1: CandleStick, cs_m2: CandleStick, param:dict):
            super().__init__(param, 'ThreeInsideDown', 'bearish', trend, [cs_m2, cs_m1, cs])
            self._relative_size = relative_size
            self._cs = cs
            self._cs_m1 = cs_m1
            self._cs_m2 = cs_m2
            self.is_pattern = self._is_three_inside_down()

        def _is_three_inside_down(self) -> bool:
            """
            method to check if the candle stick is a three inside down
            :return: bool: True if the candle stick is a three inside down, False otherwise
            """
            if self.trend_strength is None:
                return None
            if self.trend_strength >= self.param['trend_strength']:
                if self._cs_m2.type() == 'bullish' and self._cs_m2.cs_body_ratio() >= self.param['cs_m2_body_ratio'] and self._relative_size[-3] >= self.param['cs_m2_relative_size']:
                    if self._cs_m1.type() == 'bearish' and self._cs_m1.open <= self._cs_m2.close:
                        if self._cs.type() == 'bearish' and self._cs.cs_body_ratio() >= self.param['cs_body_ratio'] and self._relative_size[-1] >= self.param['cs_relative_size'] and self._cs.open >= self._cs_m1.close:
                            return True
            return False

    # BearishHarami (21)
    class BearishHarami(PatternTemplate):
        def __init__(self, trend: float, relative_size: list, cs: CandleStick, cs_m1: CandleStick, param:dict):
            super().__init__(param, 'BearishHarami', 'bearish', trend, [cs_m1, cs])
            self._relative_size = relative_size
            self._cs = cs
            self._cs_m1 = cs_m1
            self.is_pattern = self._is_bearish_harami()

        def _is_bearish_harami(self) -> bool:
            """
            method to check if the candle stick is a bearish harami
            :return: bool: True if the candle stick is a bearish harami, False otherwise
            """
            if self.trend_strength is None:
                return None
            if self.trend_strength >= self.param['trend_strength']:
                if self._cs_m1.type() == 'bullish' and self._cs_m1.cs_body_ratio() >= self.param['cs_m1_body_ratio'] and self._relative_size[-2] >= self.param['cs_m1_relative_size']:
                    if self._cs.type() == 'bearish' and self._cs.cs_body_ratio() >= self.param['cs_body_ratio'] and self._relative_size[-1] <= self.param['cs_relative_size'] and self._cs.close >= self._cs_m1.open and self._cs.open <= self._cs_m1.close:
                        return True
            return False

    # Tweezer Top (22)
    class TweezerTop(PatternTemplate):
        def __init__(self, trend: float, relative_size: list, cs: CandleStick, cs_m1: CandleStick, param:dict):
            super().__init__(param, 'TweezerTop', 'bearish', trend, [cs_m1, cs])
            self._relative_size = relative_size
            self._cs = cs
            self._cs_m1 = cs_m1
            self.is_pattern = self._is_tweezer_top()

        def _is_tweezer_top(self) -> bool:
            """
            method to check if the candle stick is a tweezer top
            :return: bool: True if the candle stick is a tweezer top, False otherwise
            """
            if self.trend_strength is None:
                return None
            if self.trend_strength >= self.param['trend_strength']:
                if self._cs_m1.type() == 'bullish' and self._cs_m1.cs_body_ratio() >= self.param['cs_m1_body_ratio'] and self._relative_size[-2] >= self.param['cs_m1_relative_size']:
                    if self._cs.type() == 'bearish' and self._cs.body_position() >= self.param['cs_body_position'] and self._cs.cs_body_ratio() <= self.param['cs_body_ratio']:
                        return True
            return False


    # Bullish Reversal Candlestick Patterns methods:
    # Hammer (1)
    def is_hammer(self, param:dict = None, is_boolean = False) -> list:
        if param is None:
            param = Parameter.candle_stick_pattern['bullish']['hammer']
        result = []
        for i in range(len(self.candle_stick_frame)):
            trend = self.trend(i, param['trend_window'])
            if is_boolean:
                result.append(self.Hammer(trend, self.candle_stick_frame[i], param).is_pattern())
            else:
                result.append(self.Hammer(trend, self.candle_stick_frame[i], param))
        return result

    # Piercing (2)
    def is_piercing(self, param:dict = None, is_boolean = False) -> list:
        if param is None:
            param = Parameter.candle_stick_pattern['bullish']['piercing']
        result = []
        for i in range(len(self.candle_stick_frame)):
            trend = self.trend(i-1, param['trend_window'])
            if is_boolean:
                result.append(self.Piercing(trend, self.candle_stick_frame[i], self.candle_stick_frame[i-1], param).is_pattern())
            else:
                result.append(self.Piercing(trend, self.candle_stick_frame[i], self.candle_stick_frame[i-1], param))
        return result

    # Bullish Engulfing (3)
    def is_bullish_engulfing(self, param:dict = None, is_boolean = False) -> list:
        if param is None:
            param = Parameter.candle_stick_pattern['bullish']['bullish_engulfing']
        result = []
        for i in range(len(self.candle_stick_frame)):
            trend = self.trend(i-1, param['trend_window'])
            if is_boolean:
                result.append(self.BullishEngulfing(trend, self.candle_stick_frame[i], self.candle_stick_frame[i-1], param).is_pattern())
            else:
                result.append(self.BullishEngulfing(trend, self.candle_stick_frame[i], self.candle_stick_frame[i-1], param))
        return result

    # Morning Star (4)
    def is_morning_star(self, param:dict = None, is_boolean = False) -> list:
        if param is None:
            param = Parameter.candle_stick_pattern['bullish']['morning_star']
        result = []
        for i in range(len(self.candle_stick_frame)):
            trend = self.trend(i-2, param['trend_window'])
            relative_size = self.min_max_body(i, param['relative_size_window'])
            if is_boolean:
                result.append(self.MorningStar(trend, relative_size, self.candle_stick_frame[i], self.candle_stick_frame[i-1], self.candle_stick_frame[i-2], param).is_pattern())
            else:
                result.append(self.MorningStar(trend, relative_size, self.candle_stick_frame[i], self.candle_stick_frame[i-1], self.candle_stick_frame[i-2], param))
        return result

    # Tree White Soldiers (5)
    def is_three_white_soldiers(self, param:dict = None, is_boolean = False) -> list:
        if param is None:
            param = Parameter.candle_stick_pattern['bullish']['three_white_soldiers']
        result = []
        for i in range(len(self.candle_stick_frame)):
            trend = self.trend(i-2, param['trend_window'])
            relative_size = self.min_max_body(i, param['relative_size_window'])
            if is_boolean:
                result.append(self.ThreeWhiteSoldiers(trend, relative_size, self.candle_stick_frame[i], self.candle_stick_frame[i-1], self.candle_stick_frame[i-2], param).is_pattern())
            else:
                result.append(self.ThreeWhiteSoldiers(trend, relative_size, self.candle_stick_frame[i], self.candle_stick_frame[i-1], self.candle_stick_frame[i-2], param))
        return result

    # Bullish Marubozu (6)
    def is_bullish_marubozu(self, param:dict = None, is_boolean = False) -> list:
        if param is None:
            param = Parameter.candle_stick_pattern['bullish']['bullish_marubozu']
        result = []
        for i in range(len(self.candle_stick_frame)):
            trend = self.trend(i, param['trend_window'])
            relative_size = self.min_max_body(i, param['relative_size_window'])
            if is_boolean:
                result.append(self.BullishMarubozu(trend, relative_size, self.candle_stick_frame[i], param).is_pattern())
            else:
                result.append(self.BullishMarubozu(trend, relative_size, self.candle_stick_frame[i], param))
        return result

    # Tree Inside Up (7)
    def is_three_inside_up(self, param:dict = None, is_boolean = False) -> list:
        if param is None:
            param = Parameter.candle_stick_pattern['bullish']['three_inside_up']
        result = []
        for i in range(len(self.candle_stick_frame)):
            trend = self.trend(i-2, param['trend_window'])
            relative_size = self.min_max_body(i, param['relative_size_window'])
            if is_boolean:
                result.append(self.ThreeInsideUp(trend, relative_size, self.candle_stick_frame[i], self.candle_stick_frame[i-1], self.candle_stick_frame[i-2], param).is_pattern())
            else:
                result.append(self.ThreeInsideUp(trend, relative_size, self.candle_stick_frame[i], self.candle_stick_frame[i-1], self.candle_stick_frame[i-2], param))
        return result

    # Bullish Harami (8)
    def is_bullish_harami(self, param:dict = None, is_boolean = False) -> list:
        if param is None:
            param = Parameter.candle_stick_pattern['bullish']['bullish_harami']
        result = []
        for i in range(len(self.candle_stick_frame)):
            trend = self.trend(i-1, param['trend_window'])
            relative_size = self.min_max_body(i, param['relative_size_window'])
            if is_boolean:
                result.append(self.BullishHarami(trend, relative_size, self.candle_stick_frame[i], self.candle_stick_frame[i-1], param).is_pattern())
            else:
                result.append(self.BullishHarami(trend, relative_size, self.candle_stick_frame[i], self.candle_stick_frame[i-1], param))
        return result

    # Tweezer Bottom (9)
    def is_tweezer_bottom(self, param:dict = None, is_boolean = False) -> list:
        if param is None:
            param = Parameter.candle_stick_pattern['bullish']['tweezer_bottom']
        result = []
        for i in range(len(self.candle_stick_frame)):
            trend = self.trend(i, param['trend_window'])
            relative_size = self.min_max_body(i, param['relative_size_window'])
            if is_boolean:
                result.append(self.TweezerBottom(trend, relative_size, self.candle_stick_frame[i], self.candle_stick_frame[i-1], param).is_pattern())
            else:
                result.append(self.TweezerBottom(trend, relative_size, self.candle_stick_frame[i], self.candle_stick_frame[i-1], param))
        return result


    # Bearish Reversal Candlestick Patterns Classes:
    # Hanging Man (14)
    def is_hanging_man(self, param:dict = None, is_boolean = False) -> list:
        if param is None:
            param = Parameter.candle_stick_pattern['bearish']['hanging_man']
        result = []
        for i in range(len(self.candle_stick_frame)):
            trend = self.trend(i, param['trend_window'])
            if is_boolean:
                result.append(self.HangingMan(trend, self.candle_stick_frame[i], param).is_pattern())
            else:
                result.append(self.HangingMan(trend, self.candle_stick_frame[i], param))
        return result

    # Dark Cloud Cover (15)
    def is_dark_cloud(self, param:dict = None, is_boolean = False) -> list:
        if param is None:
            param = Parameter.candle_stick_pattern['bearish']['dark_cloud']
        result = []
        for i in range(len(self.candle_stick_frame)):
            trend = self.trend(i-1, param['trend_window'])
            if is_boolean:
                result.append(self.DarkCloud(trend, self.candle_stick_frame[i], self.candle_stick_frame[i-1], param).is_pattern())
            else:
                result.append(self.DarkCloud(trend, self.candle_stick_frame[i], self.candle_stick_frame[i-1], param))
        return result

    # Bearish Engulfing (16)
    def is_bearish_engulfing(self, param:dict = None, is_boolean = False) -> list:
        if param is None:
            param = Parameter.candle_stick_pattern['bearish']['bearish_engulfing']
        result = []
        for i in range(len(self.candle_stick_frame)):
            trend = self.trend(i-1, param['trend_window'])
            if is_boolean:
                result.append(self.BearishEngulfing(trend, self.candle_stick_frame[i], self.candle_stick_frame[i-1], param).is_pattern())
            else:
                result.append(self.BearishEngulfing(trend, self.candle_stick_frame[i], self.candle_stick_frame[i-1], param))
        return result

    # Evening Star (17)
    def is_evening_star(self, param:dict = None, is_boolean = False) -> list:
        if param is None:
            param = Parameter.candle_stick_pattern['bearish']['evening_star']
        result = []
        for i in range(len(self.candle_stick_frame)):
            trend = self.trend(i-1, param['trend_window'])
            relative_size = self.min_max_body(i, param['relative_size_window'])
            if is_boolean:
                result.append(self.EveningStar(trend, relative_size, self.candle_stick_frame[i], self.candle_stick_frame[i-1], self.candle_stick_frame[i-2], param).is_pattern())
            else:
                result.append(self.EveningStar(trend, relative_size, self.candle_stick_frame[i], self.candle_stick_frame[i-1], self.candle_stick_frame[i-2], param))
        return result

    # Three Black Crows (18)
    def is_three_black_crows(self, param:dict = None, is_boolean = False) -> list:
        if param is None:
            param = Parameter.candle_stick_pattern['bearish']['three_black_crows']
        result = []
        for i in range(len(self.candle_stick_frame)):
            trend = self.trend(i-1, param['trend_window'])
            relative_size = self.min_max_body(i, param['relative_size_window'])
            if is_boolean:
                result.append(self.ThreeBlackCrows(trend, relative_size, self.candle_stick_frame[i], self.candle_stick_frame[i-1], self.candle_stick_frame[i-2], param).is_pattern())
            else:
                result.append(self.ThreeBlackCrows(trend, relative_size, self.candle_stick_frame[i], self.candle_stick_frame[i-1], self.candle_stick_frame[i-2], param))
        return result

    # Bearish Marubozu (19)
    def is_bearish_marubozu(self, param:dict = None, is_boolean = False) -> list:
        if param is None:
            param = Parameter.candle_stick_pattern['bearish']['bearish_marubozu']
        result = []
        for i in range(len(self.candle_stick_frame)):
            trend = self.trend(i-1, param['trend_window'])
            relative_size = self.min_max_body(i, param['relative_size_window'])
            if is_boolean:
                result.append(self.BearishMarubozu(trend, relative_size, self.candle_stick_frame[i], param).is_pattern())
            else:
                result.append(self.BearishMarubozu(trend, relative_size, self.candle_stick_frame[i], param))
        return result

    # Three Inside Down (20)
    def is_three_inside_down(self, param:dict = None, is_boolean = False) -> list:
        if param is None:
            param = Parameter.candle_stick_pattern['bearish']['three_inside_down']
        result = []
        for i in range(len(self.candle_stick_frame)):
            trend = self.trend(i-1, param['trend_window'])
            relative_size = self.min_max_body(i, param['relative_size_window'])
            if is_boolean:
                result.append(self.ThreeInsideDown(trend, relative_size, self.candle_stick_frame[i], self.candle_stick_frame[i-1], self.candle_stick_frame[i-2], param).is_pattern())
            else:
                result.append(self.ThreeInsideDown(trend, relative_size, self.candle_stick_frame[i], self.candle_stick_frame[i-1], self.candle_stick_frame[i-2], param))
        return result

    # Bearish Harami (21)
    def is_bearish_harami(self, param:dict = None, is_boolean = False) -> list:
        if param is None:
            param = Parameter.candle_stick_pattern['bearish']['bearish_harami']
        result = []
        for i in range(len(self.candle_stick_frame)):
            trend = self.trend(i-1, param['trend_window'])
            relative_size = self.min_max_body(i, param['relative_size_window'])
            if is_boolean:
                result.append(self.BearishHarami(trend, relative_size, self.candle_stick_frame[i], self.candle_stick_frame[i-1], param).is_pattern())
            else:
                result.append(self.BearishHarami(trend, relative_size, self.candle_stick_frame[i], self.candle_stick_frame[i-1], param))
        return result

    # Tweezer Top (22)
    def is_tweezer_top(self, param:dict = None, is_boolean = False) -> list:
        if param is None:
            param = Parameter.candle_stick_pattern['bearish']['tweezer_top']
        result = []
        for i in range(len(self.candle_stick_frame)):
            trend = self.trend(i-1, param['trend_window'])
            relative_size = self.min_max_body(i, param['relative_size_window'])
            if is_boolean:
                result.append(self.TweezerTop(trend, relative_size, self.candle_stick_frame[i], self.candle_stick_frame[i-1], param).is_pattern())
            else:
                result.append(self.TweezerTop(trend, relative_size, self.candle_stick_frame[i], self.candle_stick_frame[i-1], param))
        return result

In [6]:
# load data
df = pd.read_csv('EURUSD_Daily.csv')
df

,<DATE>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<TICKVOL>,<VOL>,<SPREAD>
0,2006.01.02,1.18490,1.18690,1.18010,1.18210,5870,0.000000e+00,20
1,2006.01.03,1.18220,1.20330,1.18100,1.20150,9455,0.000000e+00,20
2,2006.01.04,1.20140,1.21460,1.20120,1.21080,10123,0.000000e+00,20
3,2006.01.05,1.21070,1.21230,1.20650,1.20970,9584,0.000000e+00,20
4,2006.01.06,1.21010,1.21820,1.20780,1.21430,9219,0.000000e+00,20
...,...,...,...,...,...,...,...,...
4406,2022.12.23,1.05934,1.06323,1.05861,1.06150,762777,1.144170e+11,5
4407,2022.12.26,1.06196,1.06359,1.06140,1.06335,224659,3.369885e+10,5
4408,2022.12.27,1.06324,1.06689,1.06110,1.06385,757183,1.135770e+11,5
4409,2022.12.28,1.06385,1.06738,1.06058,1.06115,742676,1.114010e+11,5


In [7]:
# create candlestick frame
candle_stick_frame = CandleStickFrame(df['<DATE>'], df['<OPEN>'], df['<HIGH>'], df['<LOW>'], df['<CLOSE>'], df['<VOL>'])
len(candle_stick_frame)

4411

In [8]:
a = CandleStickPattern(candle_stick_frame)

param = dict(trend_window = 10,
             relative_size_window = 11,
             trend_strength = 0.00,
             cs_m1_body_ratio = 0.50,
             cs_body_ratio = 0.50,
             cs_m1_relative_size = 0.30,
             cs_body_position = 0.25,
             )

b = a.is_tweezer_bottom(param = param, is_boolean = False)

counter = 0
for c in b:
    if c.is_pattern:
        counter += 1
print(counter)

117


In [9]:
candle_pattern = CandleStickPattern(candle_stick_frame)
df['is_bullish_marubozu'] = candle_pattern.is_bullish_marubozu()
df['is_bearish_marubozu'] = candle_pattern.is_bearish_marubozu()
df['is_morning_star'] = candle_pattern.is_morning_star()
df['is_evening_star'] = candle_pattern.is_evening_star()
df['is_bullish_engulfing'] = candle_pattern.is_bullish_engulfing()
df['is_bearish_engulfing'] = candle_pattern.is_bearish_engulfing()
df['is_hammer'] = candle_pattern.is_hammer()
df['is_hanging_man'] = candle_pattern.is_hanging_man()
df['is_piercing'] = candle_pattern.is_piercing()
df['is_dark_cloud'] = candle_pattern.is_dark_cloud()
df['is_three_white_soldiers'] = candle_pattern.is_three_white_soldiers()
df['is_three_black_crows'] = candle_pattern.is_three_black_crows()
df['is_three_inside_up'] = candle_pattern.is_three_inside_up()
df['is_three_inside_down'] = candle_pattern.is_three_inside_down()
df['is_bullish_harami'] = candle_pattern.is_bullish_harami()
df['is_bearish_harami'] = candle_pattern.is_bearish_harami()
df['is_tweezer_bottom'] = candle_pattern.is_tweezer_bottom()
df['is_tweezer_top'] = candle_pattern.is_tweezer_top()
df

,<DATE>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<TICKVOL>,<VOL>,<SPREAD>,is_bullish_marubozu,is_bearish_marubozu,...,is_piercing,is_dark_cloud,is_three_white_soldiers,is_three_black_crows,is_three_inside_up,is_three_inside_down,is_bullish_harami,is_bearish_harami,is_tweezer_bottom,is_tweezer_top
0,2006.01.02,1.18490,1.18690,1.18010,1.18210,5870,0.000000e+00,20,BullishMarubozu -> (None),BearishMarubozu -> (None),...,Piercing -> (None),DarkCloud -> (None),ThreeWhiteSoldiers -> (None),ThreeBlackCrows -> (None),ThreeInsideUp -> (None),ThreeInsideDown -> (None),BullishHarami -> (None),BearishHarami -> (None),TweezerBottom -> (None),TweezerTop -> (None)
1,2006.01.03,1.18220,1.20330,1.18100,1.20150,9455,0.000000e+00,20,BullishMarubozu -> (None),BearishMarubozu -> (None),...,Piercing -> (None),DarkCloud -> (None),ThreeWhiteSoldiers -> (None),ThreeBlackCrows -> (None),ThreeInsideUp -> (None),ThreeInsideDown -> (None),BullishHarami -> (None),BearishHarami -> (None),TweezerBottom -> (None),TweezerTop -> (None)
2,2006.01.04,1.20140,1.21460,1.20120,1.21080,10123,0.000000e+00,20,BullishMarubozu -> (None),BearishMarubozu -> (None),...,Piercing -> (None),DarkCloud -> (None),ThreeWhiteSoldiers -> (None),ThreeBlackCrows -> (None),ThreeInsideUp -> (None),ThreeInsideDown -> (None),BullishHarami -> (None),BearishHarami -> (None),TweezerBottom -> (None),TweezerTop -> (None)
3,2006.01.05,1.21070,1.21230,1.20650,1.20970,9584,0.000000e+00,20,BullishMarubozu -> (None),BearishMarubozu -> (None),...,Piercing -> (None),DarkCloud -> (None),ThreeWhiteSoldiers -> (None),ThreeBlackCrows -> (None),ThreeInsideUp -> (None),ThreeInsideDown -> (None),BullishHarami -> (None),BearishHarami -> (None),TweezerBottom -> (None),TweezerTop -> (None)
4,2006.01.06,1.21010,1.21820,1.20780,1.21430,9219,0.000000e+00,20,BullishMarubozu -> (None),BearishMarubozu -> (None),...,Piercing -> (None),DarkCloud -> (None),ThreeWhiteSoldiers -> (None),ThreeBlackCrows -> (None),ThreeInsideUp -> (None),ThreeInsideDown -> (None),BullishHarami -> (None),BearishHarami -> (None),TweezerBottom -> (None),TweezerTop -> (None)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4406,2022.12.23,1.05934,1.06323,1.05861,1.06150,762777,1.144170e+11,5,BullishMarubozu -> (False),BearishMarubozu -> (False),...,Piercing -> (False),DarkCloud -> (False),ThreeWhiteSoldiers -> (False),ThreeBlackCrows -> (False),ThreeInsideUp -> (False),ThreeInsideDown -> (False),BullishHarami -> (False),BearishHarami -> (False),TweezerBottom -> (False),TweezerTop -> (False)
4407,2022.12.26,1.06196,1.06359,1.06140,1.06335,224659,3.369885e+10,5,BullishMarubozu -> (False),BearishMarubozu -> (False),...,Piercing -> (False),DarkCloud -> (False),ThreeWhiteSoldiers -> (False),ThreeBlackCrows -> (False),ThreeInsideUp -> (False),ThreeInsideDown -> (False),BullishHarami -> (False),BearishHarami -> (False),TweezerBottom -> (False),TweezerTop -> (False)
4408,2022.12.27,1.06324,1.06689,1.06110,1.06385,757183,1.135770e+11,5,BullishMarubozu -> (False),BearishMarubozu -> (False),...,Piercing -> (False),DarkCloud -> (False),ThreeWhiteSoldiers -> (False),ThreeBlackCrows -> (False),ThreeInsideUp -> (False),ThreeInsideDown -> (False),BullishHarami -> (False),BearishHarami -> (False),TweezerBottom -> (False),TweezerTop -> (False)
4409,2022.12.28,1.06385,1.06738,1.06058,1.06115,742676,1.114010e+11,5,BullishMarubozu -> (False),BearishMarubozu -> (False),...,Piercing -> (False),DarkCloud -> (False),ThreeWhiteSoldiers -> (False),ThreeBlackCrows -> (False),ThreeInsideUp -> (False),ThreeInsideDown -> (False),BullishHarami -> (False),BearishHarami -> (False),TweezerBottom -> (False),TweezerTop -> (False)


In [10]:
df['is_piercing'][409].pattern_type

'bullish'

In [62]:
import itertools
import tqdm


class CandelStickPatternFinder(CandleStickPattern):
    def __init__(self, candle_stick_frame):
        super().__init__(candle_stick_frame)
        self.patterns = list()

    def _get_n_cs_pattern(self, n_cs, trend_window, permutation, ts_confirm):
        cs_body_ratio = list()
        cs_relative_size = list()
        cs_body_position = list()
        body_ls_ratio = list()
        body_us_ratio = list()

        for n in permutation:
            cs_body_ratio.append(n[0])
            cs_relative_size.append(n[1])
            cs_body_position.append(n[2])
            body_ls_ratio.append(n[3])
            body_us_ratio.append(n[4])

        bearish_results = list()
        bullish_results = list()
        for i in range(len(self.candle_stick_frame)):
            trend = self.trend(i-n_cs, trend_window)
            last_n_cs = self.candle_stick_frame[i-n_cs:i]
            boolean_results = list()
            for n in range(len(last_n_cs)):
                try:
                    if last_n_cs[n].body_position() >= cs_body_position[n] and \
                            last_n_cs[n].cs_body_ratio() >= cs_body_ratio[n] and \
                            last_n_cs[n].cs_size() >= cs_relative_size[n] * self._average_candle_stick_size and \
                            last_n_cs[n].body_lower_shadow_ratio() >= body_ls_ratio[n] and \
                            last_n_cs[n].body_upper_shadow_ratio() >= body_us_ratio[n]:
                        boolean_results.append(True)
                    else:
                        boolean_results.append(False)
                except:
                    boolean_results.append(False)

            if trend == 'up' and all(boolean_results):
                bearish_results.append(True)
                bullish_results.append(False)
            elif trend == 'down' and all(boolean_results):
                bullish_results.append(True)
                bearish_results.append(False)
            else:
                bearish_results.append(False)
                bullish_results.append(False)
        confirmations = self._evaluate_cs_pattern((bearish_results, bullish_results), ts_confirm)
        return confirmations

    def _evaluate_cs_pattern(self, results, ts_confirm):
        bearish_confirmation_counter = 0
        bullish_confirmation_counter = 0
        for i in range(len(self.candle_stick_frame)):
            if results[0][i] is True:
                if self.trend(i-ts_confirm, ts_confirm) == 'down':
                    bearish_confirmation_counter += 1
            elif results[1][i] is True:
                if self.trend(i-ts_confirm, ts_confirm) == 'up':
                    bullish_confirmation_counter += 1
        return bearish_confirmation_counter, bullish_confirmation_counter


    def find_n_cs_pattern(self,
                          n_cs=1,
                          trend_window=10,
                          cs_body_ratio=[(0, 1, 0.2)],
                          cs_relative_size=[(0, 1, 0.2)],
                          cs_body_position=[(-1, 1, 0.2)],
                          body_ls_ratio=[(0, 1, 0.2)],
                          body_us_ratio=[(0, 1, 0.2)],
                          ts_confirm=10,
                          ):
        permutations = list()
        for n in range(n_cs):
            # permutation of all parametes with the given bordes and step size
            permutations.append(list(itertools.product(*[np.arange(*cs_body_ratio[n]),
                                                         np.arange(*cs_relative_size[n]),
                                                         np.arange(*cs_body_position[n]),
                                                         np.arange(*body_ls_ratio[n]),
                                                         np.arange(*body_us_ratio[n])])))

        # combine all permutations
        permutations = list(itertools.product(*permutations))
        print(len(permutations))

        for permutation in tqdm.tqdm(permutations):
            confirmations = self._get_n_cs_pattern(n_cs, trend_window, permutation, ts_confirm)
            self.patterns.append(('bearish', permutation, confirmations[0]))
            self.patterns.append(('bullish', permutation, confirmations[1]))
        return self.patterns

In [63]:
#save the df
df.to_csv('EURUSD_Daily_with_patterns.csv', index=False)

In [11]:
CandelStickPatternFinder(candle_stick_frame).find_n_cs_pattern(n_cs=1)

200000


  0% 744/200000 [01:55<8:33:25,  6.47it/s] 


KeyboardInterrupt: 